<a href="https://colab.research.google.com/github/aniketinamdar/Twitter-Sentiment-Analysis-using-Support-Vector-Machine-/blob/main/bidirectionalstackedlstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download en_core_web_sm

2023-03-17 06:15:14.584041: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 06:15:15.443408: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-17 06:15:15.443536: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-17 06:15:15.443554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import re
import spacy
import numpy as np
import pandas as pd
import en_core_web_sm
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
import nltk
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from spacy.lang.en.stop_words import STOP_WORDS
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/Twitter_Data.csv',nrows=10000)

In [ ]:
df['category'].fillna(0, inplace=True)

In [ ]:
df.clean_text = df.clean_text.fillna('')

In [ ]:
nlp = en_core_web_sm.load()
lemmatizer = WordNetLemmatizer()
stopwords = STOP_WORDS
EMOJI_PATTERN = re.compile(
    "["
    u"U0001F600-U0001F64F"  # emoticons
    u"U0001F300-U0001F5FF"  # symbols & pictographs
    u"U0001F680-U0001F6FF"  # transport & map symbols
    u"U0001F1E0-U0001F1FF"  # flags (iOS)
    u"U00002702-U000027B0"
    u"U000024C2-U0001F251"
    "]+", flags=re.UNICODE
  )
FILTERS = '!"#$%&()*+,-/:;?@[\]^_`{|}~tn'
HTML_TAG_PATTERN = re.compile(r']*>')
NUMBERING_PATTERN = re.compile('d+(?:st|[nr]d|th)')
DISABLE_PIPELINES = ["tok2vec", "parser", "ner", "textcat", "custom", "lemmatizer"]

In [ ]:
def initial_preprocessing(text):
    tag_removed_text = HTML_TAG_PATTERN.sub('', text)
    emoji_removed_text = EMOJI_PATTERN.sub(r'', tag_removed_text)
    numberings_removed_text =  NUMBERING_PATTERN.sub('', emoji_removed_text)
    extra_chars_removed_text = re.sub(
        r"(.)1{2,}",  r'11', numberings_removed_text
    )
    return extra_chars_removed_text

In [ ]:
def preprocess_text(doc):
    tokens = [
        token
        for token in doc
        if not token.is_space and
           not token.like_email and
           not token.like_url and
           not token.is_stop and
           not token.is_punct and
           not token.like_num
    ]
    translation_table = str.maketrans('', '', FILTERS)
    translated_tokens = [
        token.text.lower().translate(translation_table)
        for token in tokens
    ]
    lemmatized_tokens = [
        lemmatizer.lemmatize(token)
        for token in translated_tokens
        if len(token) > 1
    ]
    return lemmatized_tokens

In [ ]:
df.shape

(10000, 2)

In [ ]:
labels = df['category'].iloc[:10000]
data = df.iloc[:10000, :]
column = 'clean_text'
not_null_data = data[data[column].notnull()]
not_null_data[column] = not_null_data[column].apply(initial_preprocessing)
texts = [
    preprocess_text(doc)
    for doc in nlp.pipe(not_null_data[column], disable=DISABLE_PIPELINES)
]

In [ ]:
tokenizer = Tokenizer(
    filters=FILTERS,
    lower=True
)
padding = 'post'
tokenizer.fit_on_texts(texts)
vocab_size = len(tokenizer.word_index) + 1
sequences = []
max_sequence_len = 0
for text in texts:
    txt_to_seq = tokenizer.texts_to_sequences([text])[0]
    sequences.append(txt_to_seq)
    txt_to_seq_len = len(txt_to_seq)
    if txt_to_seq_len > max_sequence_len:
        max_sequence_len = txt_to_seq_len
padded_sequences = pad_sequences(
    sequences, 
    maxlen=max_sequence_len, 
    padding=padding
)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=max_sequence_len)) # update input length to match actual sequence length
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
adam = Adam(learning_rate=0.01)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    optimizer=adam, 
    metrics=['accuracy']
)
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 39, 64)            1088000   
                                                                 
 bidirectional_10 (Bidirecti  (None, 39, 128)          66048     
 onal)                                                           
                                                                 
 dropout_15 (Dropout)        (None, 39, 128)           0         
                                                                 
 bidirectional_11 (Bidirecti  (None, 64)               41216     
 onal)                                                           
                                                                 
 dropout_16 (Dropout)        (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 64)               

In [ ]:
padded_sequences.shape

(10000, 39)

In [ ]:
labels.shape

(10000,)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   clean_text  10000 non-null  object
 1   category    10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [ ]:
history = model.fit(
    padded_sequences, 
    labels, 
    epochs=100,
    verbose=1,
    batch_size=64)

Epoch 1/100


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


157/157 [==============================] - 35s 127ms/step - loss: 0.0184 - accuracy: 0.3781
Epoch 2/100
157/157 [==============================] - 8s 53ms/step - loss: -726.7120 - accuracy: 0.4454
Epoch 3/100
157/157 [==============================] - 5s 29ms/step - loss: -5943.0127 - accuracy: 0.5205
Epoch 4/100
157/157 [==============================] - 4s 28ms/step - loss: -18971.3926 - accuracy: 0.5261
Epoch 5/100
157/157 [==============================] - 4s 29ms/step - loss: -41043.0312 - accuracy: 0.5571
Epoch 6/100
157/157 [==============================] - 4s 21ms/step - loss: -70093.4375 - accuracy: 0.5593
Epoch 7/100
157/157 [==============================] - 3s 17ms/step - loss: -104904.8281 - accuracy: 0.5790
Epoch 8/100
157/157 [==============================] - 3s 22ms/step - loss: -145659.3594 - accuracy: 0.5728
Epoch 9/100
157/157 [==============================] - 3s 17ms/step - loss: -192034.1250 - accuracy: 0.5759
Epoch 10/100
157/157 [==============================

In [ ]:
predictions = model.predict(padded_sequences[:4])
for pred in predictions:
    print(pred[0])

1/1 [==============================] - 1s 1s/step
0.0
1.2726635e-07
1.0
1.0


In [ ]:
# Get predictions for custom input
custom_input = input("Enter a line:" )
processed_input = preprocess_text(nlp(custom_input))
custom_seq = tokenizer.texts_to_sequences([processed_input])[0]
custom_padded_seq = pad_sequences([custom_seq], maxlen=max_sequence_len, padding=padding)
custom_pred = model.predict(custom_padded_seq)

# Calculate median of all predictions
all_preds = model.predict(padded_sequences)
median_pred = np.median(all_preds)
print(median_pred)
# Compare custom prediction with median value
if custom_pred > median_pred:
    print("Positive sentiment")
else:
    print("Negative sentiment")

Enter a line:have a good day
313/313 [==============================] - 2s 6ms/step
5.3622813e-09
Positive sentiment


In [ ]:
# Split data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(
    padded_sequences, labels, test_size=0.2, random_state=42
)

# Train model on training set
history = model.fit(
    train_data,
    train_labels,
    epochs=50,
    verbose=1,
    batch_size=64
)

# Make predictions on testing set
test_predictions = model.predict(test_data)

# Convert predictions to binary labels
test_predictions = np.where(test_predictions > np.median(predictions), 1, 0)


Epoch 1/50
125/125 [==============================] - 2s 13ms/step - loss: -58386528.0000 - accuracy: 0.5390
Epoch 2/50
125/125 [==============================] - 2s 14ms/step - loss: -59500068.0000 - accuracy: 0.5385
Epoch 3/50
125/125 [==============================] - 2s 14ms/step - loss: -60896556.0000 - accuracy: 0.5409
Epoch 4/50
125/125 [==============================] - 2s 16ms/step - loss: -60506620.0000 - accuracy: 0.5332
Epoch 5/50
125/125 [==============================] - 2s 13ms/step - loss: -60918264.0000 - accuracy: 0.5420
Epoch 6/50
125/125 [==============================] - 1s 11ms/step - loss: -62138132.0000 - accuracy: 0.5362
Epoch 7/50
125/125 [==============================] - 1s 12ms/step - loss: -62731996.0000 - accuracy: 0.5282
Epoch 8/50
125/125 [==============================] - 1s 12ms/step - loss: -63733792.0000 - accuracy: 0.5290
Epoch 9/50
125/125 [==============================] - 1s 12ms/step - loss: -63820172.0000 - accuracy: 0.5272
Epoch 10/50
125/125

In [ ]:

# Calculate accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print("Accuracy:", accuracy)


Accuracy: 0.4965
